Langgraph LATS testing

Implement LATS for each of the step.
Have a generic LATS class.
First, have LATS for planning step - the system prompt should be rna_seq planner prompt.
Once the plan is solved, have LATS for each of the step.
The plan generated in the first LATS will be given to a parser. Parse each of the planning steps
Then the lats get steps one by one and the plan and each steps response will be given to the next lats class

In [24]:
import os

from dotenv import load_dotenv
load_dotenv(override=True)

True

In [25]:
# define language model
from langchain_openai import ChatOpenAI, AzureChatOpenAI

llm = AzureChatOpenAI(
        azure_deployment="gpt-4-128k",
        openai_api_version=os.getenv("AZURE_0125_MODEL_VERSION"),
        temperature=0,
        azure_endpoint=os.getenv("AZURE_0125_MODEL_ENDPOINT"),
        openai_api_key=os.getenv("AZURE_0125_MODEL_API_KEY"),
    )

In [26]:
# define tools 
from langchain.agents import load_tools
from langchain_community.tools import VectorStoreQATool
from langchain.tools import (BaseTool, StructuredTool)

from code_intrepreter_sanbox import E2B_class
from vector_db import get_vectorstore_retriever_tool


def get_tools(langsmith_run_id: str, sync=True):
      search = load_tools(["serpapi"])
      code_execution_class = E2B_class(langsmith_run_id=langsmith_run_id)
      e2b_code_execution_tool = StructuredTool.from_function(
          func=code_execution_class.run_python_code,
          name="Python-Code-Execution",
          description="Executes Python3 code in an safe Docker container.",
      )
      e2b_shell_tool = StructuredTool.from_function(
          func=code_execution_class.run_shell,
          name="Shell-commands-except-for-git",
          description=
          "Run shell commands to, for example, execute shell scripts or R scripts. It is in the same environment as the Code Execution tool.",
      )
      docs_tools: List[VectorStoreQATool] = [
          get_vectorstore_retriever_tool(
              course_name='langchain-docs',
              name='Langchain-docs',
              description=
              "Build context-aware, reasoning applications with LangChain's flexible abstractions and AI-first toolkit."),
          get_vectorstore_retriever_tool(
              course_name='ml4bio-star',
              name='STAR-docs',
              description=
              'Basic STAR workflow consists of 2 steps: (1) Generating genome indexes files and (2) Mapping reads to the genome'
          ),
          get_vectorstore_retriever_tool(
              course_name='ml4bio-fastqc',
              name='FastQC-docs',
              description=
              'FastQC aims to provide a simple way to do some quality control checks on raw sequence data coming from high throughput sequencing pipelines. It provides a modular set of analyses which you can use to give a quick impression of whether your data has any problems of which you should be aware before doing any further analysis. It works with data from BAM, SAM or FastQ files'
          ),
          get_vectorstore_retriever_tool(
              course_name='ml4bio-multiqc',
              name='MultiQC-docs',
              description=
              "MultiQC is a reporting tool that parses results and statistics from bioinformatics tool outputs, such as log files and console outputs. It helps to summarize experiments containing multiple samples and multiple analysis steps. It's designed to be placed at the end of pipelines or to be run manually when you've finished running your tools."
          ),
          get_vectorstore_retriever_tool(
              course_name='ml4bio-bioconductor',
              name='Bioconductor-docs',
              description=
              "Bioconductor is a project that contains hundreds of individual R packages. They're all high quality libraries that provide widespread access to a broad range of powerful statistical and graphical methods for the analysis of genomic data. Some of them also facilitate the inclusion of biological metadata in the analysis of genomic data, e.g. literature data from PubMed, annotation data from Entrez genes."
          ),
      ]

      tools: list[BaseTool] =  search + docs_tools + [e2b_code_execution_tool, e2b_shell_tool] 
      return tools


In [27]:
# define tools for the executor agent
import uuid
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation

langsmith_run_id = str(uuid.uuid4())  # for Langsmith
tools = get_tools(langsmith_run_id)
tool_executor = ToolExecutor(tools=tools)
tools

/Users/minum/Documents/NCSA/UIUC-Chatbot/ai-ta-backend/.venv310/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:116: UserWarning: Api key is used with unsecure connection.
  warnings.warn("Api key is used with unsecure connection.")
/Users/minum/Documents/NCSA/UIUC-Chatbot/ai-ta-backend/.venv310/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:116: UserWarning: Api key is used with unsecure connection.
  warnings.warn("Api key is used with unsecure connection.")
/Users/minum/Documents/NCSA/UIUC-Chatbot/ai-ta-backend/.venv310/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:116: UserWarning: Api key is used with unsecure connection.
  warnings.warn("Api key is used with unsecure connection.")
/Users/minum/Documents/NCSA/UIUC-Chatbot/ai-ta-backend/.venv310/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:116: UserWarning: Api key is used with unsecure connection.
  warnings.warn("Api key is used with unsecure connection.")
/Users/minum/Documen

[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='edf00a75c49d95767f0f7b99cddb763bea1145f2f94cf9f88879bbcab19c9a8f', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='edf00a75c49d95767f0f7b99cddb763bea1145f2f94cf9f88879bbcab19c9a8f', aiosession=None)>),
 VectorStoreQATool(name='Langchain-docs', description="Build context-aware, reasoning applications with LangChain's flexible abstractions and AI-first toolkit.", vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x1

Exception in thread e2b-sandbox-refresh:
Traceback (most recent call last):
  File "/Users/minum/Documents/NCSA/UIUC-Chatbot/ai-ta-backend/.venv310/lib/python3.10/site-packages/e2b/sandbox/sandbox_connection.py", line 431, in _refresh
    api.sandboxes_sandbox_id_refreshes_post(
  File "pydantic/decorator.py", line 40, in pydantic.decorator.validate_arguments.validate.wrapper_function
    from contextlib import _GeneratorContextManager
  File "pydantic/decorator.py", line 134, in pydantic.decorator.ValidatedFunction.call
    
  File "pydantic/decorator.py", line 206, in pydantic.decorator.ValidatedFunction.execute
    
  File "/Users/minum/Documents/NCSA/UIUC-Chatbot/ai-ta-backend/.venv310/lib/python3.10/site-packages/e2b/api/v1/client/api/sandboxes_api.py", line 379, in sandboxes_sandbox_id_refreshes_post
    return self.sandboxes_sandbox_id_refreshes_post_with_http_info(
  File "pydantic/decorator.py", line 40, in pydantic.decorator.validate_arguments.validate.wrapper_function
    fr

In [58]:
# generate planner - first candidate - first lats
planner_system_prompt = """
You are a world-class programmer and AI assistant capable of executing any goal related to software development, genAI, LLMs, and full-stack technologies.

First, write a step-by-step plan for the task. The plan should be descriptive and well-explained. 

The main objective is to plan and execute the workflow efficiently. Break down the execution into small, informed steps rather than attempting everything in one go.

You have access to a variety of tools, including browser, github_tools for interacting with GitHub, and multiple vectorstore instances. Utilize the browser for internet searches and github_tools for all interactions with GitHub repositories. For code execution, rely on PythonRepl and shell tools available in the Docker environment.

When you send a message containing code, it will be executed in a Docker container. You have been granted full permission to execute any code necessary to complete the task within this Docker environment using PythonRepl and shell tools as required.

When referencing files, assume they exist in the GitHub repository. Use github_tools for all interactions with GitHub and operate within the current working directory.

Communicate with the user in Markdown.

Conclude your plan with '<END_OF_PLAN>'.

Verify the solution and provide evidence where possible.

"""

planner_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", planner_system_prompt),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)


initial_planner_chain = planner_prompt_template | llm.bind_tools(tools=tools).with_config(
    run_name="GenerateInitialPlannerCandidate"
)


In [68]:
executor_system_prompt = """
You are a world-class programmer and AI assistant capable of executing any goal related to software development, genAI, LLMs, and full-stack technologies.

Write code to achieve the given task. You have access to a variety of tools, including browser, github_tools for interacting with GitHub, and multiple vectorstore instances. Utilize the browser for internet searches and github_tools for all interactions with GitHub repositories. For code execution, rely on PythonRepl and shell tools available in the Docker environment.

When you send a message containing code, it will be executed in a Docker container. You have been granted full permission to execute any code necessary to complete the task within this Docker environment using PythonRepl and shell tools as required.

Before any execution task, prepare the development environment, whether that be a notebook, .sh, .py, .ipynb, .R, or other file types. Incrementally develop, execute, and debug the code, committing changes to GitHub regularly.

For package installations, use pip, and strive to install all necessary packages in a single command. In case of failures, debug and install them correctly, adhering to the Pydantic structure to avoid validation errors.

When referencing files, assume they exist in the GitHub repository. Use github_tools for all interactions with GitHub and operate within the current working directory.

Prefer universally applicable packages that are likely to be pre-installed and compatible across different applications. For example, ffmpeg and pandoc are recommended for their broad support and functionality.

Include steps and EXACT CODE SNIPPETS if they are applicable to the task. Do not suggest code that requires user modifications, and ensure all code is complete and executable as is.

For code that needs to be saved to a file, indicate this with # filename: <filename> at the start of the code block. Only include one code block per response and avoid asking users to copy and paste results. Use the print function for outputs.

Execute your code and provide results. If an error arises, correct it and provide the updated code. 

If a solution isn't reached after successful code execution, reassess your approach, gather more information, and propose an alternative method.

Verify the solution and provide evidence where possible.

End the interaction with "TERMINATE" once the task is completed.

"""

executor_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", executor_system_prompt),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)


initial_executor_chain = executor_prompt_template | llm.bind_tools(tools=tools).with_config(
    run_name="GenerateInitialExecutorCandidate"
)


In [45]:

parser = JsonOutputToolsParser(return_id=True)

In [47]:
initial_response = initial_answer_chain.invoke(
    {"input": "Implement an RNA-Sequence Analysis Workflow using DESEQ2."}
)
initial_response

 ------- MIGHT TRUNCATE MESSAGES ------- 
message content='\nYou are a world-class programmer and AI assistant capable of executing any goal related to software development, genAI, LLMs, and full-stack technologies.\n\nFirst, write a plan. Recap the plan between each code block. This recapping is necessary to maintain the context due to the short-term memory constraints.\n\nWhen you send a message containing code, it will be executed in a Docker container. You have been granted full permission to execute any code necessary to complete the task within this Docker environment using PythonRepl and shell tools as required.\n\nYou have access to a variety of tools, including browser, github_tools for interacting with GitHub, and multiple vectorstore instances. Utilize the browser for internet searches and github_tools for all interactions with GitHub repositories. For code execution, rely on PythonRepl and shell tools available in the Docker environment.\n\nBefore any execution task, prepar

AIMessage(content='To implement an RNA-Sequence Analysis Workflow using DESeq2, we will follow a structured approach. DESeq2 is a popular R package used for analyzing count data from RNA sequencing experiments to find differentially expressed genes. The workflow will involve several steps, including quality control, alignment, counting, and differential expression analysis. Here\'s a detailed plan:\n\n1. **Environment Setup**: Ensure R and necessary packages (DESeq2, Bioconductor) are installed.\n2. **Quality Control**: Use FastQC to assess the quality of the raw sequencing data.\n3. **Alignment**: Align the reads to a reference genome using a tool like STAR or HISAT2.\n4. **Counting**: Count the number of reads that map to each gene using featureCounts or a similar tool.\n5. **Differential Expression Analysis**: Use DESeq2 to perform differential expression analysis on the count data.\n6. **Results Interpretation**: Interpret the results, focusing on significantly differentially expre

In [48]:
from __future__ import annotations

import math
from collections import deque
from typing import List, Optional

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage


class Node:
    def __init__(
        self,
        messages: List[BaseMessage],
        reflection: Reflection,
        parent: Optional[Node] = None,
    ):
        self.messages = messages
        self.parent = parent
        self.children = []
        self.value = 0
        self.visits = 0
        self.reflection = reflection
        self.depth = parent.depth + 1 if parent is not None else 1
        self._is_solved = reflection.found_solution if reflection else False
        if self._is_solved:
            self._mark_tree_as_solved()
        self.backpropagate(reflection.normalized_score)

    def __repr__(self) -> str:
        return (
            f"<Node value={self.value}, visits={self.visits},"
            f" solution={self.messages} reflection={self.reflection}/>"
        )

    @property
    def is_solved(self):
        """If any solutions exist, we can end the search."""
        return self._is_solved

    @property
    def is_terminal(self):
        return not self.children

    @property
    def best_child(self):
        """Select the child with the highest UCT to search next."""
        if not self.children:
            return None
        return max(self.children, key=lambda child: child.upper_confidence_bound())

    @property
    def best_child_score(self):
        """Return the child with the highest value."""
        if not self.children:
            return None
        return max(self.children, key=lambda child: int(child.is_solved) * child.value)

    @property
    def height(self) -> int:
        """Check for how far we've rolled out the tree."""
        if self.children:
            return 1 + max([child.height for child in self.children])
        return 1

    def upper_confidence_bound(self, exploration_weight=1.0):
        """Return the UCT score. This helps balance exploration vs. exploitation of a branch."""
        if self.parent is None:
            raise ValueError("Cannot obtain UCT from root node")
        if self.visits == 0:
            return self.value
        # Encourages exploitation of high-value trajectories
        average_reward = self.value / self.visits
        # Encourages exploration of less-visited trajectories
        exploration_term = math.sqrt(math.log(self.parent.visits) / self.visits)
        return average_reward + exploration_weight * exploration_term

    def backpropagate(self, reward: float):
        """Update the score of this node and its parents."""
        node = self
        while node:
            node.visits += 1
            node.value = (node.value * (node.visits - 1) + reward) / node.visits
            node = node.parent

    def get_messages(self, include_reflections: bool = True):
        if include_reflections:
            return self.messages + [self.reflection.as_message()]
        return self.messages

    def get_trajectory(self, include_reflections: bool = True) -> List[BaseMessage]:
        """Get messages representing this search branch."""
        messages = []
        node = self
        while node:
            messages.extend(
                node.get_messages(include_reflections=include_reflections)[::-1]
            )
            node = node.parent
        # Reverse the final back-tracked trajectory to return in the correct order
        return messages[::-1]  # root solution, reflection, child 1, ...

    def get_best_solution(self):
        """Return the best solution from within the current sub-tree."""
        all_nodes = [self]
        nodes = deque()
        nodes.append(self)
        while nodes:
            node = nodes.popleft()
            all_nodes.extend(node.children)
            for n in node.children:
                nodes.append(n)
        best_node = max(
            all_nodes,
            # We filter out all non-terminal, non-solution trajectories
            key=lambda node: int(node.is_terminal and node.is_solved) * node.value,
        )
        return best_node
    
    def _get_all_children(self):
        all_nodes = []
        nodes = deque()
        nodes.append(self)
        while nodes:
            node = nodes.popleft()
            all_nodes.extend(node.children)
            for n in node.children:
                nodes.append(n)
        return all_nodes

    def _mark_tree_as_solved(self):
        parent = self.parent
        while parent:
            parent._is_solved = True
            parent = parent.parent

In [78]:
# graph state

from typing_extensions import TypedDict


class PlannerTreeState(TypedDict):
    # The full tree
    root: Node
    # The original input
    input: str


class ExecutorTreeState(TypedDict):
    # The full tree
    root: Node
    # The plan input - one step at a time
    input: str
    previous_steps: Annotated[List[str], operator.add] = []
    previous_results: Annotated[List[str], operator.add] = []


# after each plan execution, clean the tree
# executortree.clear()

        

In [50]:
# Define the planner node we will add to the graph
def generate_initial_planner_response(state: PlannerTreeState) -> dict:
    """Generate the initial candidate response."""
    res = initial_planner_chain.invoke({"input": state["input"]})
    parsed = parser.invoke(res)
    tool_responses = tool_executor.batch(
        [ToolInvocation(tool=r["type"], tool_input=r["args"]) for r in parsed]
    )
    output_messages = [res] + [
        ToolMessage(content=json.dumps(resp), tool_call_id=tool_call["id"])
        for resp, tool_call in zip(tool_responses, parsed)
    ]
    reflection = reflection_chain.invoke(
        {"input": state["input"], "candidate": output_messages}
    )
    root = Node(output_messages, reflection=reflection)
    return {
        **state,
        "root": root,
    }

In [51]:
# Define the executor node we will add to the graph
def generate_initial_executor_response(state: ExecutorTreeState) -> dict:
    """Generate the initial candidate response."""
    res = initial_executor_chain.invoke({"input": state["input"]})
    state.previous_steps.append(state["input"])
    parsed = parser.invoke(res)
    tool_responses = tool_executor.batch(
        [ToolInvocation(tool=r["type"], tool_input=r["args"]) for r in parsed]
    )
    output_messages = [res] + [
        ToolMessage(content=json.dumps(resp), tool_call_id=tool_call["id"])
        for resp, tool_call in zip(tool_responses, parsed)
    ]
    reflection = reflection_chain.invoke(
        {"input": state["input"], "candidate": output_messages}
    )
    root = Node(output_messages, reflection=reflection)
    return {
        **state,
        "root": root,
    }

In [52]:
# This generates N candidate values
# for a single input to sample actions from the environment
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables import RunnableConfig


def generate_candidates(messages: ChatPromptValue, config: RunnableConfig):
    n = config["configurable"].get("N", 5)
    bound_kwargs = llm.bind_tools(tools=tools).kwargs
    print("bound kwargs")
    print(bound_kwargs)
    chat_result = llm.generate(
        [messages.to_messages()],
        n=n,
        callbacks=config["callbacks"],
        run_name="GenerateCandidates",
        **bound_kwargs
    )
    print("chat result : ", chat_result)
    return [gen.message for gen in chat_result.generations[0]]


expansion_chain = prompt_template | generate_candidates

In [53]:
# Candidate generation node
# We will package the candidate generation and reflection steps in the following "expand" node. 

from collections import defaultdict, deque


def plannerexpand(state: PlannerTreeState, config: RunnableConfig) -> dict:
    """Starting from the "best" node in the tree, generate N candidates for the next step."""
    print("RunnableConfig :", config)
    print("RunnableConfig configurable:", config["configurable"])
    root = state["root"]
    best_candidate: Node = root.best_child if root.children else root
    messages = best_candidate.get_trajectory()
    print("messages :")
    print(messages)
    # Generate N candidates from the single child candidate
    new_candidates = expansion_chain.invoke(
        {"input": state["input"], "messages": messages}, config
    )
    parsed = parser.batch(new_candidates)
    flattened = [
        (i, tool_call)
        for i, tool_calls in enumerate(parsed)
        for tool_call in tool_calls
    ]
    tool_responses = tool_executor.batch(
        [
            ToolInvocation(tool=tool_call["type"], tool_input=tool_call["args"])
            for _, tool_call in flattened
        ]
    )
    collected_responses = defaultdict(list)
    for (i, tool_call), resp in zip(flattened, tool_responses):
        collected_responses[i].append(
            ToolMessage(content=json.dumps(resp), tool_call_id=tool_call["id"])
        )
    output_messages = []
    for i, candidate in enumerate(new_candidates):
        output_messages.append([candidate] + collected_responses[i])
    print("candidate output messages")
    print(output_messages)
    # Reflect on each candidate
    # For tasks with external validation, you'd add that here.
    reflections = reflection_chain.batch(
        [{"input": state["input"], "candidate": msges} for msges in output_messages],
        config,
    )
    # Grow tree
    child_nodes = [
        Node(cand, parent=best_candidate, reflection=reflection)
        for cand, reflection in zip(output_messages, reflections)
    ]
    print("child nodes")
    print(child_nodes)
    best_candidate.children.extend(child_nodes)
    # We have already extended the tree directly, so we just return the state
    return state

In [73]:
# Candidate generation node
# We will package the candidate generation and reflection steps in the following "expand" node. 

from collections import defaultdict, deque


def executorexpand(state: ExecutorTreeState, config: RunnableConfig) -> dict:
    """Starting from the "best" node in the tree, generate N candidates for the next step."""
    print("RunnableConfig :", config)
    print("RunnableConfig configurable:", config["configurable"])
    root = state["root"]
    best_candidate: Node = root.best_child if root.children else root
    messages = best_candidate.get_trajectory()
    print("messages :")
    print(messages)
    # Generate N candidates from the single child candidate
    new_candidates = expansion_chain.invoke(
        {"input": state["current_step"], "messages": messages}, config
    )
    parsed = parser.batch(new_candidates)
    flattened = [
        (i, tool_call)
        for i, tool_calls in enumerate(parsed)
        for tool_call in tool_calls
    ]
    tool_responses = tool_executor.batch(
        [
            ToolInvocation(tool=tool_call["type"], tool_input=tool_call["args"])
            for _, tool_call in flattened
        ]
    )
    collected_responses = defaultdict(list)
    for (i, tool_call), resp in zip(flattened, tool_responses):
        collected_responses[i].append(
            ToolMessage(content=json.dumps(resp), tool_call_id=tool_call["id"])
        )
    output_messages = []
    for i, candidate in enumerate(new_candidates):
        output_messages.append([candidate] + collected_responses[i])
    print("candidate output messages")
    print(output_messages)
    # Reflect on each candidate
    # For tasks with external validation, you'd add that here.
    reflections = reflection_chain.batch(
        [{"input": state["input"], "candidate": msges} for msges in output_messages],
        config,
    )
    # Grow tree
    child_nodes = [
        Node(cand, parent=best_candidate, reflection=reflection)
        for cand, reflection in zip(output_messages, reflections)
    ]
    print("child nodes")
    print(child_nodes)
    best_candidate.children.extend(child_nodes)
    print("best_candidate")
    print(best_candidate)
    state["previous_steps"].append(state["current_step"])
    state["previous_results"].append(best_candidate.messages[-1])
    # We have already extended the tree directly, so we just return the state
    return state

In [55]:
# create graph
# With those two nodes defined, we are ready to define the graph. After each agent step, we have the option of finishing.

from langgraph.graph import END, StateGraph


def should_loop(state: TreeState):
    """Determine whether to continue the tree search."""
    root = state["root"]
    if root.is_solved:
        return END
    if root.height > 5:
        return END
    return "expand"


planner_builder = StateGraph(PlannerTreeState)
planner_builder.add_node("start", generate_initial_planner_response)
planner_builder.add_node("expand", plannerexpand)
planner_builder.set_entry_point("start")


planner_builder.add_conditional_edges(
    "start",
    # Either expand/rollout or finish
    should_loop,
)
planner_builder.add_conditional_edges(
    "expand",
    # Either continue to rollout or finish
    should_loop,
)

planner_graph = planner_builder.compile()

In [59]:
# invoke

question = "Implement an RNA-Sequence Analysis Workflow using DESeq2."
for step in planner_graph.stream({"input": question}):
    step_name, step_state = next(iter(step.items()))
    print(step_name)
    print("rolled out: ", step_state["root"].height)
    print("---")

 ------- MIGHT TRUNCATE MESSAGES ------- 
message content="\nYou are a world-class programmer and AI assistant capable of executing any goal related to software development, genAI, LLMs, and full-stack technologies.\n\nFirst, write a step-by-step plan for the task. The plan should be descriptive and well-explained. \n\nThe main objective is to plan and execute the workflow efficiently. Break down the execution into small, informed steps rather than attempting everything in one go.\n\nYou have access to a variety of tools, including browser, github_tools for interacting with GitHub, and multiple vectorstore instances. Utilize the browser for internet searches and github_tools for all interactions with GitHub repositories. For code execution, rely on PythonRepl and shell tools available in the Docker environment.\n\nWhen you send a message containing code, it will be executed in a Docker container. You have been granted full permission to execute any code necessary to complete the task w

In [60]:
planner_solution_node = step["__end__"]["root"].get_best_solution()
planner_best_trajectory = planner_solution_node.get_trajectory(include_reflections=False)
print(planner_best_trajectory[-1].content)

To implement an RNA-Sequence Analysis Workflow using DESeq2, we will follow a structured approach. DESeq2 is a method for differential gene expression analysis based on the negative binomial distribution. It's widely used in bioinformatics for analyzing count data from RNA sequencing experiments. The workflow will involve several steps, from quality control of the raw data to the final differential expression analysis using DESeq2. Here's a step-by-step plan:

### Step 1: Quality Control of Raw Data
- **1.1.** Use FastQC to perform initial quality checks on the raw sequencing data.
- **1.2.** Based on FastQC reports, decide if any preprocessing (like trimming) is needed.

### Step 2: Preprocessing and Cleaning
- **2.1.** If necessary, use tools like Trimmomatic or Cutadapt to trim adapters and low-quality bases.
- **2.2.** Optionally, use FastQC again to check the quality of the data post-trimming.

### Step 3: Alignment
- **3.1.** Align the cleaned reads to the reference genome using 

In [80]:
executor_builder = StateGraph(ExecutorTreeState)
executor_builder.add_node("start", generate_initial_executor_response)
executor_builder.add_node("expand", executorexpand)
executor_builder.set_entry_point("start")


executor_builder.add_conditional_edges(
    "start",
    # Either expand/rollout or finish
    should_loop,
)
executor_builder.add_conditional_edges(
    "expand",
    # Either continue to rollout or finish
    should_loop,
)

executor_graph = executor_builder.compile()

In [63]:
executor_plan_step = """
### Step 1: Quality Control of Raw Data
- **1.1.** Use FastQC to perform initial quality checks on the raw sequencing data.
- **1.2.** Based on FastQC reports, decide if any preprocessing (like trimming) is needed.
"""

In [81]:
# invoke executor
for step in executor_graph.stream({"input": executor_plan_step}):
    step_name, step_state = next(iter(step.items()))
    print(step_name)
    print("rolled out: ", step_state["root"].height, step_state["root"])
    print("---")

 ------- MIGHT TRUNCATE MESSAGES ------- 
message content='\nYou are a world-class programmer and AI assistant capable of executing any goal related to software development, genAI, LLMs, and full-stack technologies.\n\nWrite code to achieve the given task. You have access to a variety of tools, including browser, github_tools for interacting with GitHub, and multiple vectorstore instances. Utilize the browser for internet searches and github_tools for all interactions with GitHub repositories. For code execution, rely on PythonRepl and shell tools available in the Docker environment.\n\nWhen you send a message containing code, it will be executed in a Docker container. You have been granted full permission to execute any code necessary to complete the task within this Docker environment using PythonRepl and shell tools as required.\n\nBefore any execution task, prepare the development environment, whether that be a notebook, .sh, .py, .ipynb, .R, or other file types. Incrementally deve

In [70]:
executor_solution_node = step["__end__"]["root"].get_best_solution()
executor_best_trajectory = executor_solution_node.get_trajectory(include_reflections=False)
print(executor_best_trajectory[-1].content)

To perform the initial quality checks on the raw sequencing data using FastQC, we'll simulate the process since we can't directly run FastQC in this environment. However, I'll guide you through the steps you would take and provide an example of how to interpret the results from FastQC reports to decide if any preprocessing, such as trimming, is needed.

### Step 1.1: Using FastQC for Initial Quality Checks

Normally, you would run FastQC on your raw sequencing data files (usually in FASTQ format) using the following command in a terminal:

```bash
fastqc your_data_file.fastq
```

This command generates a report in HTML format that you can view in any web browser. The report contains several sections, each providing insights into different aspects of your data quality.

### Step 1.2: Interpreting FastQC Reports for Preprocessing Decisions

Here are some key sections of the FastQC report to pay attention to for deciding on preprocessing:

1. **Per base sequence quality**: This plot shows